## Road Segmentation 
using fully convolutional network <br>
and pretrained VGG-16 model


In [23]:
import glob
from imutils import paths
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import torch
from torch import nn
from torch.optim import SGD, Adam
import numpy as np
import torchvision.transforms as transforms
from torchvision import models
import cv2
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision.utils import make_grid

In [24]:
# load data
train_dir = glob.glob("./data/data_road/training/image_2/*.png")
train_gt_dir = glob.glob("./data/data_road/training/gt_image_2/*.png")
test_dir = glob.glob("./data/data_road/testing/image_2/*.png")

image_dataset_path = "./data/data_road/training/image_2/"
mask_dataset_path = "./data/data_road/training/gt_image_2/"

In [25]:
class RoadSegmentationDataset(Dataset):
    def __init__(self, image_paths, target_paths, transforms):
        self.image_paths = image_paths
        self.target_paths = target_paths
        self.transforms = transforms
        
    def __getitem__(self, idx):
        image = cv2.imread(self.image_paths[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.target_paths[idx])
        
        if self.transforms is not None:
            image = self.transforms(image)
            mask = self.transforms(mask)
        return (image, mask)
    
    def __len__(self):
        return len(self.image_paths)

In [26]:
im_path = sorted(list(paths.list_images(image_dataset_path)))
mask_path = sorted(list(paths.list_images(mask_dataset_path)))

split = train_test_split(im_path, mask_path, test_size=0.2, random_state=42)

(train_im, test_im) = split[:2]
(train_mask, test_mask) = split[2:]

# write the testing image paths to disk so that we can use then
# when evaluating/testing our model
#print("[INFO] saving testing image paths...")
#f = open(config.TEST_PATHS, "w")
#f.write("\n".join(testImages))
#f.close()


transforms = transforms.Compose([transforms.ToPILImage(), 
                                 transforms.Resize((224,224)),
                                 transforms.functional.rgb_to_grayscale,
                                 transforms.ToTensor()])

train_dataset = RoadSegmentationDataset(train_im, train_mask, transforms)
test_dataset = RoadSegmentationDataset(test_im, test_mask, transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [27]:
def plot_2(image, mask):
    image = image.permute(1,2,0)
    mask = mask.permute(1,2,0)
    fig = plt.figure(figsize=(12,12))
    fig.add_subplot(1,2,1)
    plt.imshow(image)
    plt.axis("off")
    fig.add_subplot(1,2,2)
    plt.imshow(mask)
    plt.axis("off")

In [28]:
# first batch
train_features, train_mask = next(iter(train_loader))
print(f"Features Batch Shape: {train_features.size()}")
print(f"Labels Batch Shape: {train_mask.size()}")
img = train_features[2].squeeze()
mask = train_mask[2]

Features Batch Shape: torch.Size([32, 1, 224, 224])
Labels Batch Shape: torch.Size([32, 1, 224, 224])


In [29]:
plot_2(img,mask)

RuntimeError: number of dims don't match in permute

### VGG-16

In [30]:
vgg16 = models.vgg16(pretrained=True)
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [31]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


Freeze convolutional layer 

In [32]:
for param in vgg16.features.parameters():
    param.requires_grad = False

In [33]:
# change the classifier of vgg16 to concolutional layer and upsample

class FCN32(nn.Module):
  def __init__(self):
    super(FCN32, self).__init__()
    self.features = vgg16.features
    self.classifier = nn.Sequential(
      nn.Conv2d(512, 4096, 7),
      nn.ReLU(inplace=True),
      #nn.Dropout2d(),
      nn.Conv2d(4096, 4096, 1),
      nn.ReLU(inplace=True),
      #nn.Dropout2d(),
      nn.Conv2d(4096, 21, 1),
      nn.ConvTranspose2d(21, 21, 224, stride=32)
    )

  def forward(self, x):
    x = self.features(x)
    x = self.classifier(x)
    return x

fcn = FCN32()
fcn.to(device)


FCN32(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation

In [34]:
print('FEATURE parameters requires gradients: {}'.format(any([param.requires_grad for param in fcn.features.parameters()])))
print('CLASSIFIER parameters requires gradients: {}'.format(all([param.requires_grad for param in fcn.classifier.parameters()])))
print('Transferred FEATURE parameters from VGG16 to FCN: {}'.format(list(fcn.features.parameters()) == list(vgg16.features.parameters())))
print('Number of params transfered: {}'.format(sum(p.numel() for p in fcn.features.parameters())))
print('Number of params left in network to be updated: {}'.format(sum(p.numel() for p in fcn.classifier.parameters())))

FEATURE parameters requires gradients: False
CLASSIFIER parameters requires gradients: True
Transferred FEATURE parameters from VGG16 to FCN: True
Number of params transfered: 14714688
Number of params left in network to be updated: 141759530


In [35]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(fcn.parameters(), lr=0.0005)

In [36]:
def get_mean_iou(conf_mat, multiplier=1.0):
    cm = conf_mat.copy()
    np.fill_diagonal(cm, np.diag(cm) * multiplier)
    inter = np.diag(cm)
    gt_set = cm.sum(axis=1)
    pred_set = cm.sum(axis=0)
    union_set =  gt_set + pred_set - inter
    iou = inter.astype(float) / union_set
    mean_iou = np.nanmean(iou)
    return mean_iou

In [37]:
best_loss = 1000000000
j = 0
for epoch in range(15):
    val_running_loss = 0
    iou_running_score = 0
    dice_running_score = 0
    fcn.train()
    for i, dat in enumerate(train_loader):
        j += 1
        # Get image, label pair
        inputs, labels = dat

        # Using GPU
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Set parameter gradients to 0
        optimizer.zero_grad()

        # Forward pass for a batch
        outputs = fcn(inputs)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backpropagate
        loss.backward()

        # Update the weights
        optimizer.step()

        # print training loss
        print('[{}, {}] training loss:{}'.format(epoch+1, i+1, loss.item()))

    fcn.eval()
    with torch.no_grad():
        for k, dat1 in enumerate(test_loader):
            # Get image, label pair
            inputs1, labels1 = dat1

            # Using GPU
            inputs1 = inputs1.to(device)
            labels1 = labels1.to(device)

            # Predicting segmentation for val inputs
            outputs1 = fcn(inputs1)

            # Compute CE loss and aggregate it
            loss1 = criterion(outputs1, labels1)
            val_running_loss += loss1

            # Reshaping prediction segmentations and actual segmentations for iou and dice score
            preds = torch.argmax(outputs1, dim=1).detach().cpu().numpy()
            gt = labels1.detach().cpu().numpy()

            # Compute confusion matrix
            conf_mat = confusion_matrix(y_pred=preds.flatten(), y_true=gt.flatten(), labels=list(range(21)))

            # Computing iou and dice scores and aggregating them
            iou_score = get_mean_iou(conf_mat=conf_mat)
            iou_running_score += iou_score
            dice_score = get_mean_iou(conf_mat=conf_mat, multiplier=2.0)
            dice_running_score += dice_score

        # Averaging loss and scores
        avg_val_loss = float(val_running_loss)/(k+1)
        avg_iou_score = float(iou_running_score)/(k+1)
        avg_dice_score = float(dice_running_score)/(k+1)

        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            torch.save(fcn.state_dict(), '/content/best_model_fcn16.pth.tar')

        # Visualizations for batch wise metrics
        print('epoch {}, validation loss: {}, iou score: {}, dice score: {}'.format(epoch+1, avg_val_loss, avg_iou_score, avg_dice_score))
    
print('Finished Training')

RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[32, 1, 224, 224] to have 3 channels, but got 1 channels instead